In [1]:
import IPython.display as ipd

# An overview in pratice of AWS's application services for machine learning

This notebook demonstrates how to use AWS's application services for machine learning tasks like image and text analysis. 

## Initialization
For this demo, we've prepared images that are hosted on Amazon S3.

In [2]:
import boto3
import json

aws_region = 'us-east-1'
s3_bucket = "fx-public-samples"

sample_scene = "images/jakarta_skyline.jpg"
sample_meme = "images/obama_aws_meme.png"

rekognition = boto3.client('rekognition', region_name=aws_region)
translate = boto3.client('translate', region_name=aws_region)
comprehend = boto3.client('comprehend', region_name=aws_region)
polly = boto3.client('polly')


# A basic exemple of image analysis

![alt text](https://s3.amazonaws.com/fx-public-samples/images/jakarta_skyline.jpg "Logo Title Text 1")

# Object and scene detection

In [3]:
def rekognitionLabel(bucket, key, confidence=70):
    response = rekognition.detect_labels(Image={
        'S3Object': {
            'Bucket': bucket,
            'Name': key,
            }
        },
        MinConfidence=confidence
    )
    return response

In [4]:
response = rekognitionLabel(s3_bucket, sample_scene, 95)
print(json.dumps(response["Labels"], indent=4, sort_keys=True))

[
    {
        "Confidence": 98.32048034667969, 
        "Name": "Building"
    }, 
    {
        "Confidence": 98.32048034667969, 
        "Name": "City"
    }, 
    {
        "Confidence": 98.32048034667969, 
        "Name": "Town"
    }, 
    {
        "Confidence": 98.32048034667969, 
        "Name": "Urban"
    }
]


# A more complexe exemple

![alt text](https://s3.amazonaws.com/fx-public-samples/images/obama_aws_meme.png "Logo Title Text 1")

## Celebrity Detection

In [5]:
def rekognitionCelebrity(bucket, key):
    response = rekognition.recognize_celebrities(Image={
        'S3Object': {
            'Bucket': bucket,
            'Name': key,
            }
        }
    )
    return response

In [6]:
response = rekognitionCelebrity(s3_bucket, sample_meme)

celebrities = [k["Name"] for k in response["CelebrityFaces"]]
print("Response: " + json.dumps(celebrities, indent=4, sort_keys=True))

Response: [
    "Madelyn Dunham"
]


## Text in image

In [7]:
def rekognitionText(bucket, key):
    response = rekognition.detect_text(Image={
        'S3Object': {
            'Bucket': bucket,
            'Name': key,
            }
        }
    )
    return response

In [8]:
response = rekognitionText(s3_bucket, sample_meme)

lines = [k["DetectedText"] for k in response["TextDetections"] if k['Type'] == 'LINE']
print("Number of lines: {}".format(len(lines)))

text = " ".join(lines)
print("Response: " + text)

Number of lines: 3
Response: I'M PROUD OF YOU THAT MOMENT WHEN YOU DEPLOY YOUR VERY FIRST AWS LAMBDA FUNCTION USING GO


# Working with text

Let's use Amazon Comprehend our natural language processing (NLP) service that uses machine learning to find insights and relationships in text.

## Language Detection
The Language Detection API automatically identifies text written in over 100 languages and returns the dominant language with a confidence score to support that a language is dominant.

In [9]:
def languageDetection(text):
    language = comprehend.detect_dominant_language(
        Text=text
    )['Languages'][0]['LanguageCode']
    return language

In [10]:
language = languageDetection(text)
print("Language: " + language)

Language: en


## Sentiment Detection

The Sentiment Analysis API returns the overall sentiment of a text (Positive, Negative, Neutral, or Mixed).

In [11]:
def sentimentDetection(text):
    sentiment = comprehend.detect_sentiment(
        Text=text,
        LanguageCode=language
    )
    return sentiment

In [12]:
sentiment = sentimentDetection(text)
print("Sentiment: " + json.dumps(sentiment["Sentiment"], indent=4, sort_keys=True))

Sentiment: "POSITIVE"


## Key Phrases Detection

The Keyphrase Extraction API returns the key phrases or talking points and a confidence score to support that this is a key phrase.

In [13]:
def keyPhrasesDetection(text):
    response = comprehend.detect_key_phrases(
        Text=text,
        LanguageCode=language
    )
    key_phrases = [k["Text"] for k in response["KeyPhrases"] ]
    return key_phrases

In [14]:
key_phrases = keyPhrasesDetection(text)
print("Key phrases: " + json.dumps(key_phrases, indent=4))

Key phrases: [
    "PROUD", 
    "THAT MOMENT", 
    "YOUR VERY FIRST AWS LAMBDA FUNCTION"
]


## Translation
Amazon Translate uses deep learning models to deliver more accurate and more natural sounding translation than traditional statistical and rule-based translation algorithms.

In [15]:
result = translate.translate_text(Text=text,
                                  SourceLanguageCode="en",
                                  TargetLanguageCode="fr")
print(result['TranslatedText'])

JE SUIS FIER DE VOUS CE MOMENT OÙ VOUS DÉPLOYEZ VOTRE TOUTE PREMIÈRE FONCTION AWS LAMBDA À L'AIDE DE GO


## Text-to-speech
Amazon Polly is a Text-to-Speech service that uses advanced deep learning technologies to synthesize speech that sounds like a human voice.

In [16]:
response = polly.synthesize_speech(OutputFormat='mp3',
                                   Text=text,
                                   VoiceId='Matthew')

audio = response['AudioStream'].read()

In [17]:
ipd.Audio(audio)

# Find me on GitHub

![alt text](https://s3.amazonaws.com/fx-public-samples/QRCode-AWS-API-Services.png "Logo Title Text 1")

https://github.com/fxgsell/aws-ai-api-services